# Sistemas Inteligentes 2020/2021

## Mini-projeto 3: Aprendizagem Automática

## Relatório


## Grupo: 32

Número: 53354 - Nome: João Viana (4)

Número: 53747 - Nome: João Nunes (4)

Número: 53749 - Nome: Pedro Rodrigues (4)

## 1. Processamento dos dados






## Dataset

O conjunto de dados apresentado é nos fornecido pelo heart_1.txt e tem apenas como objetivo o ajustamento de um modelo e seleção de um modelo de aprendizagem que garanta os melhores resultados na previsão da presença de uma doença cardio-vascular. Este conjunto de dados é constituido por 245 pessoas. Existem 14 atributos, 3 deles binários (sexo, açúcar no sangue em jejum> 120 mg / dl, angina induzida por exercício) e 10 que não são binários (idade,tipo de dor no peito,pressão arterial em repouso, colesterol sérico em mg / dl, resultados eletrocardiográficos de repouso, frequência cardíaca máxima alcançada, pico antigo = depressão ST induzida por exercício em relação ao repouso, a inclinação do segmento ST de pico do exercício,número de vasos principais (0-3) coloridos por fluorosopia,thal: 3 = normal; 6 = defeito corrigido; 7 = defeito reversível). O atributo target corresponde ao **target** e indica a existência de doença cardiovascular (1), ou não (0). 


In [88]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

heart= pd.read_csv("heart_1.txt", sep="\t")
heart





,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
2,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
3,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
4,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,63,0,0,124,197,0,1,136,1,0.0,1,0,2,0
241,59,1,0,164,176,1,0,90,0,1.0,1,2,1,0
242,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
243,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [89]:
X = heart.values[:,(0,1,2,3,4,5,6,7,8,9,10,11,12)]
Y = heart.values[:,13]
col_names = np.array(heart.columns)[[0,1,2,3,4,5,6,7,8,9,10,11,12]]

## Seleção de Variáveis

In [90]:
def entropy(y):
    values, counts = np.unique(y,return_counts=True)
    probs=counts/len(y)
    #print(counts, probs)
    return -np.sum(probs*np.log2(probs))


def select_var(X, y):
    N,M = X.shape
    min_ent=1e34
    best_var=-1
    for v in range(M):
        ent=0
        for val in np.unique(X[:,v]):
            srows = X[:,v] == val
            ent+=np.sum(srows)/N*entropy(y[srows])
        if ent<min_ent:
            min_ent=ent
            best_var=v
    return best_var

 
X = heart.values[:,(0,1,2,3,4,5,6,7,8,9,10,11,12)]
Y = heart.values[:,13]
best_var = select_var(X, Y)

print("A melhor variável segundo os valores de entropia:" + col_names[best_var])


A melhor variável segundo os valores de entropia:chol


## 2. Ajustamento dos modelos


### 2.1. Teste de modelos e validação




## Árvore de Decisão


In [91]:
class SI_DecisionTree():
    def __init__(self):
        
        self.tree=None
        
    def select_var(self, X, y):
        N,M = X.shape
        min_ent=1e34
        best_var=-1
        for v in range(M):
            ent=0
            for val in np.unique(X[:,v]):
                srows = X[:,v] == val
                ent+=np.sum(srows)/N*entropy(y[srows])
            if ent<min_ent:
                min_ent=ent
                best_var=v
        return best_var
    
    def decision_tree(self, X, y, col_names, the_path):
        sv=self.select_var(X, y)
        N,M=X.shape
        if M==1:
            if len(np.unique(y))==1:
                the_path.append(str(y[0]))
            else:
                #isto acontece quando o nó não é puro!
                #devolve o valor mais comum (em caso de empate devolve o primeiro que encontra)
                values, counts = np.unique(y,return_counts=True)
                the_path.append(str(values[counts.argmax()]))
        else:
            X0=np.delete(X, sv, 1)
            cnames=np.delete(col_names,sv)
            for val in np.unique(X[:,sv]):
                srows = X[:,sv]==val
                X_new=X0[srows, :]
                y_new=y[srows]
                the_path.append([(col_names[sv], val)])
                if len(np.unique(y_new))>1:
                    self.decision_tree(X_new, y_new, cnames, the_path[-1] )
                else:
                    the_path[-1].append(str(y_new[0]))
        return the_path
    
    def show_tree(self, tr, level=0):
        for branch in tr:
            if type(branch[1])==str and not isinstance(branch, tuple):
                print("\t"*(level), "%s = %s --> %s" % (branch[0][0], branch[0][1], branch[1]))
            elif type(branch[1])==list:
                print("\t"*level,  "%s = %s" % (branch[0][0], branch[0][1]))
                self.show_tree(branch, level+1)

    def make_prediction(self, D, tree):
        for branch in tree:
            if type(branch[1])==str and not isinstance(branch, tuple): 
                if D[branch[0][0]]== branch[0][1]: return branch[1]
            elif type(branch[1])==list:
                if D[branch[0][0]]== branch[0][1]: 
                    return self.make_prediction(D, branch)
    
    def display(self):
        if self.tree is None: 
            print("A árvore nãoe está treinada")
            return
        self.show_tree(self.tree)

    def fit(self, X, y, col_names=None):
        N,M=X.shape
        if col_names is None: col_names=list(range(M))
        self.col_names=col_names[:]
        self.tree = self.decision_tree(X, y, col_names, [])
        return self

    def predict(self, data):
        preds=[]
        for elem in data:
            D={col: elem[i] for i, col in enumerate(self.col_names)}
            preds.append(self.make_prediction(D, self.tree))

        return preds

    
print("ARVORES DE DECISAO")
print("-----------------------------------------")

dt = SI_DecisionTree()
dt.fit(X,Y, col_names)
dt.display()
print("-----------------------------------------")

print("Arvore de decisão sem max profundidade")
dt1 = DecisionTreeClassifier(criterion = "entropy")

X1_train, X1_test, Y1_train, Y1_test = train_test_split(X,Y, test_size = 0.2, random_state = 22)
dt1.fit(X1_train,Y1_train)
preds = dt1.predict(X1_test)



print("A accuracy é: %7.4f" % accuracy_score(Y1_test, preds))
print("A precision é: %7.4f" % precision_score(Y1_test, preds))
print("O recall é: %7.4f" % recall_score(Y1_test, preds))
print("O score F1 é: %7.4f" % f1_score(Y1_test, preds))
print("-----------------------------------------")

print("Arvore de decisão com max profundidade 5")


dt2 = DecisionTreeClassifier(criterion = "entropy", max_depth = 5)

X2_train, X2_test, Y2_train, Y2_test = train_test_split(X,Y, test_size = 0.2, random_state = 22)
dt2.fit(X2_train,Y2_train)
preds = dt2.predict(X2_test)

print("A accuracy é: %7.4f" % accuracy_score(Y2_test, preds))
print("A precision é: %7.4f" % precision_score(Y2_test, preds))
print("O recall é: %7.4f" % recall_score(Y2_test, preds))
print("O score F1 é: %7.4f" % f1_score(Y2_test, preds))
print("-----------------------------------------")

print("Arvore de decisão com max profundidade 2")

dt3 = DecisionTreeClassifier(criterion = "entropy", max_depth = 2)

X3_train, X3_test, Y3_train, Y3_test = train_test_split(X,Y, test_size = 0.2, random_state = 22)
dt3.fit(X3_train,Y3_train)
preds = dt3.predict(X3_test)

print("A accuracy é: %7.4f" % accuracy_score(Y3_test, preds))
print("A precision é: %7.4f" % precision_score(Y3_test, preds))
print("O recall é: %7.4f" % recall_score(Y3_test, preds))
print("O score F1 é: %7.4f" % f1_score(Y3_test, preds))
print("-----------------------------------------")










ARVORES DE DECISAO
-----------------------------------------
 chol = 126.0 --> 1.0
 chol = 131.0 --> 0.0
 chol = 141.0 --> 1.0
 chol = 149.0
	 age = 49.0 --> 0.0
	 age = 71.0 --> 1.0
 chol = 157.0 --> 1.0
 chol = 160.0 --> 1.0
 chol = 164.0 --> 0.0
 chol = 167.0 --> 0.0
 chol = 172.0 --> 0.0
 chol = 174.0 --> 0.0
 chol = 175.0 --> 1.0
 chol = 176.0 --> 0.0
 chol = 177.0
	 age = 43.0 --> 0.0
	 age = 46.0 --> 1.0
	 age = 59.0 --> 0.0
	 age = 65.0 --> 1.0
 chol = 180.0 --> 1.0
 chol = 182.0 --> 1.0
 chol = 183.0 --> 1.0
 chol = 184.0 --> 0.0
 chol = 186.0 --> 1.0
 chol = 187.0 --> 0.0
 chol = 188.0 --> 0.0
 chol = 192.0 --> 1.0
 chol = 193.0
	 age = 56.0 --> 1.0
	 age = 68.0 --> 0.0
 chol = 195.0 --> 1.0
 chol = 196.0 --> 1.0
 chol = 197.0
	 age = 44.0 --> 0.0
	 age = 46.0 --> 1.0
	 age = 53.0 --> 1.0
	 age = 58.0 --> 1.0
	 age = 63.0 --> 0.0
	 age = 76.0 --> 1.0
 chol = 198.0 --> 0.0
 chol = 199.0 --> 1.0
 chol = 200.0 --> 0.0
 chol = 201.0 --> 1.0
 chol = 203.0
	 age = 41.0 --> 1.0
	 ag

## Naive Bayes

In [92]:
def SI_NB(X, y):
    #X =df_covid.drop(columns=["Covid"]).values
    N, M=X.shape

    #primeiro calcular os priors
    yv, yc=np.unique(y, return_counts=True)
    priors={yv[i]: yc[i]/sum(yc) for i in range(len(yv)) }
    #agora as verosimilhanças - têm que ser todas inicializadas a zero
    #As verosimilhanças podem ser lidas como 
    # L[i, "B" "A"] P(Xi =B | Y = A) 
    L_hoods={}
    for j in range(M):
        xs=np.unique(X[:,j])
        for v in xs:
            for yi in yv: L_hoods[(j, v,  yi)]=0.0
    #para cada classe de output
    for yi in yv:
        X_c = X[y==yi]
        #...procuramos cada coluna individual de X
        for j in range(M):
            col = X_c[:,j]
            vs, cs = np.unique(col, return_counts=True)
            # ...e para cada valor possível
            for i, v in enumerate(vs):
                L_hoods[(j, v,  yi)] = cs[i]/np.sum(cs)
    

    return priors, L_hoods



def show_NB(priors, L_hoods):
    print("Os Priors são:")
    for yi in priors:
        print("\tP(Y = %s) = %7.4f" % (yi, priors[yi]))
    print("As verosimilhanças são:")
    for i, xi, yi in L_hoods:
        print("\tP(X%d = %s | Y = %s) = %7.4f" % (i, xi, yi, L_hoods[(i,xi,yi)]))


priors, L_hoods = SI_NB(X, Y)
print("-----------------------------------------")
show_NB(priors, L_hoods)


X2_train, X2_test, Y2_train, Y2_test = train_test_split(X,Y, test_size = 0.2, random_state = 22)

mdl = CategoricalNB(alpha=1)


mdl.fit(X2_train, Y2_train)
print("-----------------------------------------")

preds = mdl.predict(X2_test)
print("O Accuracy score é: ", accuracy_score(Y2_test, preds))
print("A precision é: %7.4f" % precision_score(Y2_test, preds))
print("O recall é: %7.4f" % recall_score(Y2_test, preds))
print("O score F1 é: %7.4f" % f1_score(Y2_test, preds))
print("-----------------------------------------")








-----------------------------------------
Os Priors são:
	P(Y = 0.0) =  0.4571
	P(Y = 1.0) =  0.5429
As verosimilhanças são:
	P(X0 = 29.0 | Y = 0.0) =  0.0000
	P(X0 = 29.0 | Y = 1.0) =  0.0075
	P(X0 = 34.0 | Y = 0.0) =  0.0000
	P(X0 = 34.0 | Y = 1.0) =  0.0075
	P(X0 = 35.0 | Y = 0.0) =  0.0089
	P(X0 = 35.0 | Y = 1.0) =  0.0150
	P(X0 = 37.0 | Y = 0.0) =  0.0000
	P(X0 = 37.0 | Y = 1.0) =  0.0075
	P(X0 = 38.0 | Y = 0.0) =  0.0089
	P(X0 = 38.0 | Y = 1.0) =  0.0150
	P(X0 = 39.0 | Y = 0.0) =  0.0089
	P(X0 = 39.0 | Y = 1.0) =  0.0226
	P(X0 = 40.0 | Y = 0.0) =  0.0179
	P(X0 = 40.0 | Y = 1.0) =  0.0000
	P(X0 = 41.0 | Y = 0.0) =  0.0089
	P(X0 = 41.0 | Y = 1.0) =  0.0376
	P(X0 = 42.0 | Y = 0.0) =  0.0000
	P(X0 = 42.0 | Y = 1.0) =  0.0451
	P(X0 = 43.0 | Y = 0.0) =  0.0179
	P(X0 = 43.0 | Y = 1.0) =  0.0376
	P(X0 = 44.0 | Y = 0.0) =  0.0179
	P(X0 = 44.0 | Y = 1.0) =  0.0526
	P(X0 = 45.0 | Y = 0.0) =  0.0089
	P(X0 = 45.0 | Y = 1.0) =  0.0301
	P(X0 = 46.0 | Y = 0.0) =  0.0179
	P(X0 = 46.0 | Y = 1.0) =

## K-vizinhos

In [93]:
print("----------------")
print("Modelo K-vizinhos com 7")
knn = KNeighborsClassifier(n_neighbors = 7)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 22)

knn.fit(X_train, Y_train)

preds=knn.predict(X_test)
print("Accuracy : %7.4f" % accuracy_score(Y_test, preds))
print("Precision: %7.4f" % precision_score(Y_test, preds))
print("Recall   : %7.4f" % recall_score(Y_test, preds))
print("F1-Score : %7.4f" % f1_score(Y_test, preds))





print("----------------")
print("Após aplicação do scaler")


scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled= scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)

knn.fit(X_train_scaled, Y_train)

preds=knn.predict(X_test_scaled)
print("Accuracy : %7.4f" % accuracy_score(Y_test, preds))
print("Precision: %7.4f" % precision_score(Y_test, preds))
print("Recall   : %7.4f" % recall_score(Y_test, preds))
print("F1-Score : %7.4f" % f1_score(Y_test, preds))
print("----------------")



print("------------------")
print("Modelo K-vizinhos com 5")
knn = KNeighborsClassifier(n_neighbors = 5)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 22)

knn.fit(X_train, Y_train)

preds=knn.predict(X_test)
print("Accuracy : %7.4f" % accuracy_score(Y_test, preds))
print("Precision: %7.4f" % precision_score(Y_test, preds))
print("Recall   : %7.4f" % recall_score(Y_test, preds))
print("F1-Score : %7.4f" % f1_score(Y_test, preds))



## Em seguida aplicaremos o scaler

print("----------------")
print("Após aplicação do scaler")


scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled= scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)

knn.fit(X_train_scaled, Y_train)

preds=knn.predict(X_test_scaled)
print("Accuracy : %7.4f" % accuracy_score(Y_test, preds))
print("Precision: %7.4f" % precision_score(Y_test, preds))
print("Recall   : %7.4f" % recall_score(Y_test, preds))
print("F1-Score : %7.4f" % f1_score(Y_test, preds))
print("----------------")

print("------------------")



print("Modelo K-vizinhos com 3")
knn = KNeighborsClassifier(n_neighbors = 3)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 22)

knn.fit(X_train, Y_train)

preds=knn.predict(X_test)
print("Accuracy : %7.4f" % accuracy_score(Y_test, preds))
print("Precision: %7.4f" % precision_score(Y_test, preds))
print("Recall   : %7.4f" % recall_score(Y_test, preds))
print("F1-Score : %7.4f" % f1_score(Y_test, preds))





print("----------------")
print("Após aplicação do scaler")


scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled= scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)

knn.fit(X_train_scaled, Y_train)

preds=knn.predict(X_test_scaled)
print("Accuracy : %7.4f" % accuracy_score(Y_test, preds))
print("Precision: %7.4f" % precision_score(Y_test, preds))
print("Recall   : %7.4f" % recall_score(Y_test, preds))
print("F1-Score : %7.4f" % f1_score(Y_test, preds))
print("----------------")



----------------
Modelo K-vizinhos com 7
Accuracy :  0.5510
Precision:  0.6071
Recall   :  0.6071
F1-Score :  0.6071
----------------
Após aplicação do scaler
Accuracy :  0.8367
Precision:  0.8333
Recall   :  0.8929
F1-Score :  0.8621
----------------
------------------
Modelo K-vizinhos com 5
Accuracy :  0.5510
Precision:  0.6071
Recall   :  0.6071
F1-Score :  0.6071
----------------
Após aplicação do scaler
Accuracy :  0.8571
Precision:  0.8621
Recall   :  0.8929
F1-Score :  0.8772
----------------
------------------
Modelo K-vizinhos com 3
Accuracy :  0.5102
Precision:  0.5909
Recall   :  0.4643
F1-Score :  0.5200
----------------
Após aplicação do scaler
Accuracy :  0.7959
Precision:  0.8214
Recall   :  0.8214
F1-Score :  0.8214
----------------


### 2.2. Seleção e apresentação do melhor modelo




|MODELO|ACCURACY|PRECISION|RECALL|F1-SCORE|
|:-|:-|:-|:-|:-|
|Árvore de Decisão sem max profundidade|0.7959|0.8462|0.7857|0.8148|
|Árvore de Decisão com max profundidade 5|0.8163|0.8800|0.7857|0.8302|
|Árvore de Decisão com max profundidade 2|0.7143|0.7917|0.6786|0.7308|
|Naive Bayes|0.7959183673469388|0.8000|0.8571|0.8276|
|K-vizinhos com 7|0.8367|0.8333|0.8929|0.8621|
|K-vizinhos com 5|0.8571|0.8621|0.8929|0.8772|
|K-vizinhos com 3|0.7959|0.8214|0.8214|0.8214|




## Neste contexto é possível concluir que o K-vizinhos com 7 trata-se do melhor modelo na prevenção de doenças-cardiovasculares, uma vez que apresenta o maior valor de accuracy, significando que é o modelo que menos se engana a apresentar o diagnóstico final.


## 3. Validação do modelo final ajustado com um conjunto de validação independente

## DATASET

In [78]:

heart1= pd.read_csv("heart_ivs.txt", sep="\t")

heart1

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
1,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
2,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
3,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
4,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1
5,49,1,1,130,266,0,1,171,0,0.6,2,0,2,1
6,58,0,2,120,340,0,1,172,0,0.0,2,0,2,1
7,59,1,0,135,234,0,1,161,0,0.5,1,0,3,1
8,40,1,3,140,199,0,1,178,1,1.4,2,0,3,1
9,59,1,2,150,212,1,1,157,0,1.6,2,0,2,1


In [83]:
X1 = heart1.values[:,(0,1,2,3,4,5,6,7,8,9,10,11,12)]
Y1 = heart1.values[:,13]
col_names1 = np.array(heart1.columns)[[0,1,2,3,4,5,6,7,8,9,10,11,12]]


print("------------------")
knn1 = KNeighborsClassifier(n_neighbors = 7)

knn1.fit(X1, Y1)


preds=knn1.predict(X1)
print("Accuracy : %7.4f" % accuracy_score(Y1, preds))
print("Precision: %7.4f" % precision_score(Y1, preds))
print("Recall   : %7.4f" % recall_score(Y1, preds))
print("F1-Score : %7.4f" % f1_score(Y1, preds))

------------------
Accuracy :  0.7586
Precision:  0.7143
Recall   :  0.9375
F1-Score :  0.8108


## Após a análise dos resultados obtidos e confrontados com os resultados obtidos anteriormente chegamos à conclusão que o modelo Knn não será o  modelo mais indicado para esta situação sendo que o valor da accuracy é de 0.7586 o que significa que 2/3 dos diagnósticos estariam incorretos.  